In [4]:
!pip install clingo
!pip install clyngor-with-clingo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.4 MB/s eta 0:00:00
  Created wheel for clyngor-with-clingo: filename=clyngor_with_clingo-5.3.post1-py3-none-any.whl size=5798467 sha256=a7f485ae847475cf14cdf32ebe3139956e477417bbf9f4ce3bf9237ab6f42efb
  Stored in directory: /root/.cache/pip/wheels/4b/27/5a/6a46db2c467a62223ff36049f2fd474b5b69da156711cbfb11
  Created wheel for pyPEG2: filename=pyPEG2-2.15.2-py3-none-any.whl size=22768 sha256=8712e68bd91c3192eff25e87531a52a89814932c48fc3c9766d443d48f180116
  Stored in directory: /root/.cache/pip/wheels/97/d1/6d/624ecb76583a420c345cc1b216821a1

In [5]:

import clingo
import tempfile
from clyngor import ASP, solve

In [55]:
program1 = """

% FACTS

% Main protagonists of the story
person(p;f).

% Possible locations
location(madrid;cemetery;family).

% Current and potential situations
%situation(s1).
%situation(s2).
situation(s3).


% Belief base for the environment's objective facts and for the main agent's beliefs
beliefbase(env;p).



% Description of situation s1 (current situation)

%% Beliefs about location
believe_init(env, at(r,cemetery)) :-situation(s1).
believe_init(p, at(r,family)):-situation(s1).

%% Beliefs about hostility
believe_init(p,hostile(f)) :-situation(s1).
believe_init(env,hostile(f)):-situation(s1).
believe_init(env,murderer(f)):-situation(s1).
believe_init(p,murderer(f)):-situation(s1).

%% Beliefs about credible locations
believe_init(p,credible(at(r,madrid), f)):-situation(s1).
believe_init(p,credible(at(r,cemetery), f)):-situation(s1).
believe_init(p,credible(at(r,family), f)):-situation(s1).
believe_init(env,credible(at(r,cemetery), f)):-situation(s1).
believe_init(env,credible(at(r,family), f)):-situation(s1).

% Description of situation s2

%% Beliefs about location
believe_init(env, at(r,cemetery)) :-situation(s2).
believe_init(p, at(r,family)):-situation(s2).

%% Beliefs about credible locations
believe_init(p,credible(at(r,madrid), f)):-situation(s2).
believe_init(p,credible(at(r,cemetery), f)):-situation(s2).
believe_init(p,credible(at(r,family), f)):-situation(s2).
believe_init(env,credible(at(r,cemetery), f)):-situation(s2).
believe_init(env,credible(at(r,family), f)):-situation(s2).

% Description of situation s3

%% Beliefs about location
believe_init(env, at(r,cemetery)) :-situation(s3).
believe_init(p, at(r,family)):-situation(s3).

%% Beliefs about hostility
believe_init(p,hostile(f)) :-situation(s3).
%believe_init(env,hostile(f)):-situation(s3).
%believe_init(env,murderer(f)):-situation(s3).
believe_init(p,murderer(f)):-situation(s3).

%% Beliefs about credible locations
believe_init(p,credible(at(r,madrid), f)):-situation(s3).
believe_init(p,credible(at(r,cemetery), f)):-situation(s3).
believe_init(p,credible(at(r,family), f)):-situation(s3).
believe_init(env,credible(at(r,cemetery), f)):-situation(s3).
believe_init(env,credible(at(r,family), f)):-situation(s3).

% Relation between questioning round and actions
1{act(tell(p,at(r,L), f)):location(L);act(silence(p, f))}1.



% ----------------------------------------------------------------------------------------------------------------

% RULES FOR PHYSICAL ACTIONS
% Rule for checking a location
believe_cons(Y,check(P2,X,P1)) :- act(tell(P1, at(r,X), P2)), believe_init(Y,credible(at(r,X), P2)).


% Rule for evasion
believe_cons(Y,evade(P1,P2)) :- believe_cons(Y,check(P2,madrid,P1)).

% Rule for killing
believe_cons(Y,kill(P2,r,P1)) :- believe_cons(Y,check(P2,X,P1)), believe_init(Y,at(r,X)), believe_init(Y,hostile(P2)).

% Rule for helping
believe_cons(Y,help(P2)) :- believe_cons(Y,check(P2,X,P1)), believe_init(Y,at(r,X)), not believe_init(Y,hostile(P2)).

% Rule for agents loosing patience (first condition)
%believe_cons(Y,patienceLost(P2, P1)) :- act(tell(P1, at(r,X), P2)), not believe_init(Y,credible(at(r,X), P2)), beliefbase(Y).

% Rule for agents loosing patience (second condition)
believe_cons(Y,patienceLost(P2, P1)) :- act(silence(P1, P2)), believe_init(Y,hostile(P2)).

% Rule for wrong information provided
believe_cons(Y,wrongInfo(P2, P1)) :- believe_cons(Y,check(P2, X, P1)), not believe_init(Y,at(r,X)).

% Rule for an agent being harmed based on another agent loosing patience
believe_cons(Y,harm1(P2, P1)) :- believe_cons(Y,patienceLost(P2, P1)).

% Rule for an agent being harmed based on hostility and another agent providing wrong information
believe_cons(Y,harm2(P2, P1)) :- believe_cons(Y,wrongInfo(P2, P1)), believe_init(Y,hostile(P2)), not believe_cons(Y,evade(P1,P2)).


% ----------------------------------------------------------------------------------------------------------------

% RULES FOR SPEECH ACTS
% Rule for honest communication based on agents' telling what they believe
honest(P,F) :- act(tell(P,F,_)), believe_init(P,F).

% Rule for dishonest communication based on agents' not telling what they believe
dishonest(P,F) :- act(tell(P,F,_)), not believe_init(P,F).

% Rules for telling objective truth based on the environment's belief
truth(P,F) :- act(tell(P,F,_)), believe_init(env,F).

% Rules for telling objective falsity based on the environment's belief
falsity(P,F) :- act(tell(P,F,_)), not believe_init(env,F).

% Rules for telling an objective truth or an erroneous truth
objective_truth(P,F) :- honest(P,F), truth(P,F).
erroneous_truth(P,F) :- honest(P,F), falsity(P,F).

% Rules for telling an objective lie onAlternative  1,  Pablo  once  again  lies  to  the  hostile842Falangists  by  claiming  that  Ramon  is  in  Madrid.  Here,843unlike  in  the  original  version  of  the  story,  the  lie  helps844both  Ramon  and  Pablo  since  the  Falangists  will  search845for  Ramon  in  distant  Madrid,  giving  Pablo  enough  time846to  escape.  But  despite  its  greater  advantage,  “Madrid”847is  considered  equallypermissibleas  “Cemetery”  under848principialism.  Inprincipialism1,  the  exception  al-849lowing a lie is granted due to the hostility of the Falangists.850Underprincipialism2,  counterfactual  reasoning  shows851that  telling  the  truth  (“Ramon  is  at  his  family’s  house”)852would  lead  to  worse  outcomes  than  either  lie,  rendering853“Madrid”  and  “Cemetery”  morally  equivalent  within  the854framework  of  this  theory.  This  equivalence  does  not  hold855underconsequentialism1. From the perspective of ex-856pected utility, “Madrid” uniquely enables both Ramon’s and857Pablo’s survival, whereas “Cemetery”, though still protec-858tive of Ramon, increases the personal risk to Pablo. Thus,859whileprincipialismdeems both theobjectivelie860“Madrid”  and  theerroneouslie“Cemetery”  equally861permissible,consequentialism1identifies  “Madrid”  as862the uniquely optimal choicer an erroneous lie
objective_lie(P,F) :- dishonest(P,F), falsity(P,F).
erroneous_lie(P,F) :- dishonest(P,F), truth(P,F).


%%%%%%%%%%%%%%%  DEONTOLOGISM %%%%%%%%%%%%%%%%%%

% Maxims prohibiting lying and murder for deontologism
maxim(doNotLie;doNotEnableMurder).

% Rule concerning the violation of the maxim against lying
violated(doNotLie) :- dishonest(P,F).

% Rule concerning the violation of the maxim against murder
violated(doNotEnableMurder) :- believe_cons(P1,kill(P2,r,P1)).

% Permissibility and impermissibility for deontologism
impermissible(deontologism):- violated(R), maxim(R).
permissible(deontologism):- not impermissible(deontologism).


%%%%%%%%%%%%%%%  PRINCIPIALISM VERSION 1 %%%%%%%%%%%%%%%

% Principle prohibiting lying with exception and murder for principialism1
principle(doNotLieExcept;doNotEnableMurder).

% Rule stating violation of lying principle in the absence of first exception
violated(doNotLieExcept) :- act(tell(P,F,P2)), not believe_init(P,F), not exception(act(tell(P,F,P2)),doNotLieExcept).

% Rule stating that Falangists do not deserve truth
dontDeserveTruth(P2):- believe_init(p,murderer(P2)).

% First exception rule for lies: lying is permitted if truth is not deserved
exception(act(tell(P,F,P2)),doNotLieExcept):- act(tell(P,F,P2)), dontDeserveTruth(P2).

% Permissibility and impermissibility for principialism1
impermissible(principialism1) :- violated(R), principle(R).
permissible(principialism1) :- not impermissible(principialism1).


%%%%%%%%%%%%%%%  UTILITIES CALCULATIONS %%%%%%%%%%%%%%%%%%

% Expected utilities for various physical actions according to consequentialism
uti(kill(P2,r,P1),-4):-person(P2), person(P1).
uti(harm1(P1,P2),-2):-person(P2), person(P1).
uti(harm2(P1,P2),-4):-person(P2), person(P1).
uti(wrongInfo(P1,P2),0):-person(P2), person(P1).
uti(help(P),2):-person(P).

% Actions triggering specific utility values
trigUti(Y,F,N) :- uti(F,N), believe_cons(Y,F), beliefbase(Y).

% Calculation of total utility values for agents and actions
totalUti(Y,T):- T=#sum{N:uti(F,N),believe_cons(Y,F)}, beliefbase(Y).

#show act/1.
#show believe_init/2.
#show believe_cons/2.
#show impermissible/1.
#show permissible/1.
#show violated/1.
#show trigUti/3.
#show totalUti/2.
#show objective_lie/2.
#show objective_truth/2.
#show erroneous_lie/2.
#show erroneous_truth/2.
#show beliefbase/1.
#show principle/1.
#show dontDeserveTruth/1.

"""


In [56]:
answers = ASP(program1)

def imprimer(N, L):
    print("\nAnswer Set ", N)
    for X in L:
        print(X[0], X[1])
n = 1
for answer in answers:
    imprimer(n, answer)
    n = n+1



Answer Set  1
believe_init ('env', 'credible(at(r,cemetery),f)')
permissible ('principialism1',)
believe_init ('env', 'credible(at(r,family),f)')
believe_init ('env', 'at(r,cemetery)')
believe_init ('p', 'hostile(f)')
believe_init ('p', 'credible(at(r,cemetery),f)')
believe_init ('p', 'credible(at(r,family),f)')
violated ('doNotLie',)
totalUti ('env', 0)
objective_lie ('p', 'at(r,madrid)')
beliefbase ('p',)
totalUti ('p', 0)
principle ('doNotEnableMurder',)
believe_init ('p', 'murderer(f)')
believe_cons ('p', 'wrongInfo(f,p)')
dontDeserveTruth ('f',)
believe_cons ('p', 'evade(p,f)')
believe_init ('p', 'credible(at(r,madrid),f)')
principle ('doNotLieExcept',)
believe_cons ('p', 'check(f,madrid,p)')
trigUti ('p', 'wrongInfo(f,p)', 0)
believe_init ('p', 'at(r,family)')
impermissible ('deontologism',)
act ('tell(p,at(r,madrid),f)',)
beliefbase ('env',)

Answer Set  2
trigUti ('p', 'harm1(f,p)', -2)
believe_init ('env', 'credible(at(r,cemetery),f)')
permissible ('principialism1',)
believe_

In [57]:
#############  PRINCIPIALISM VERSION 2 ####################  +  #############  CONSEQUENTIALISM VERSIONS 1 AND 2 ####################


# Define program2
program2 = """


% PRINCIPIALISM VERSION 2
% Rule for comparing decisions based on counterfactual comparison
counterfactual(S1, S2, tell(P1, at(r, L1), P2)) :-
    act(S1,tell(P1, at(r, L1), P2)), L1!=L2,
    believe(P1, at(r, L2)),
    act(S2,tell(P1, at(r, L2), P2)).

% Second exception rule for lies: lying is permitted if truth leads to worst consequences
exceptionRel(S, doNotLieExcept) :-
    act(S,tell(p, at(r, L1), P2)),
    counterfactual(S, S2, tell(p, at(r, L1), P2)),
    totalUti(S, p, T1),
    totalUti(S2, p, T2),
    T1 > T2.

% Rule stating (local) violation of lying principle in the absence of second exception
violated(S, R) :-
    locally_violated(S, R),
    not exceptionRel(S, R).

% Permissibility and impermissibility for principialism2
permissible(principialism2, S) :- not impermissible(principialism2, S), act(S, _).
impermissible(principialism2, S) :- violated(S, R), principle(R).

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

% CONSEQUENTIALISM VERSION 1
% Permissibility and impermissibility for consequentialism1
impermissible(consequentialism1,S) :- totalUti(S,p,N1), totalUti(S2,p,N2), N2>N1.
permissible(consequentialism1,S) :- not impermissible(consequentialism1,S), act(S,_).


% CONSEQUENTIALISM VERSION 2
% Calculation of total utility based on sums of total utilities of each action
totalUti2(S,Y,T) :- T=#sum{N:totalUti(S,Y,N)}, beliefbase(Y),act(S,_).


% Permissibility and impermissibility for consequentialism2
impermissible(consequentialism2,S) :- totalUti2(S,env,N1), totalUti2(S2,env,N2), N2>N1.
permissible(consequentialism2,S) :- not impermissible(consequentialism2,S), act(S,_).

#show totalUti2/3.
#show totalUti/3.

%#show locally_violated/2.
%#show violated/2.
#show act/2.
%#show exceptionRel/2.
%#show counterfactual/3.
#show permissible/2.
#show impermissible/2.
"""

# Function to convert answer sets to valid ASP facts
def answer_set_to_facts(answer_set,n,first):
    formatted_facts = []
    transmit= [("believe_init",2),("believe_cons",2),("beliefbase",1), ("principle",1)]
    augment = [("act",1),("totalUti",2),("permissible",1), ("impermissible",1)]
    renamedAug = {("violated",1):"locally_violated"}
    for fact in answer_set:
        # Handle complex terms
        name = fact[0]
        args = fact[1] if isinstance(fact[1], tuple) else (fact[1],)
        arity = len(args)
        if first and (name,arity) in transmit :
          formatted_facts.append(f"{name}({','.join(map(str, args))}).")
        if (name,arity) in augment:
          if name in {"permissible","impermissible"}:
            newargs = args + ('s'+str(n),)
          else :
            newargs = ('s'+str(n),)+args
          formatted_facts.append(f"{name}({','.join(map(str, newargs))}).")
        if (name,arity) in renamedAug:
          newargs = ('s'+str(n),)+args
          newname = renamedAug[(name,arity)]
          formatted_facts.append(f"{newname}({','.join(map(str, newargs))}).")
    return "\n".join(formatted_facts)

# Run program1 to get its answer sets
answers_program1 = ASP(program1)
combined_program = ""

# Process each answer set from program1
for n, answer_set in enumerate(answers_program1, start=1):
    print(f"Processing Answer Set {n} of program1...")

    # Convert answer set to ASP facts
    facts_from_program1 = answer_set_to_facts(answer_set,n,(n==1))

    # Combine program1 and program2
    combined_program = combined_program+"%AS"+str(n)+"\n"+f"{facts_from_program1}\n"

combined_program=combined_program+"\n"+program2
print(combined_program)

Processing Answer Set 1 of program1...
Processing Answer Set 2 of program1...
Processing Answer Set 3 of program1...
Processing Answer Set 4 of program1...
%AS1
believe_init(env,credible(at(r,cemetery),f)).
permissible(principialism1,s1).
believe_init(env,credible(at(r,family),f)).
believe_init(env,at(r,cemetery)).
believe_init(p,hostile(f)).
believe_init(p,credible(at(r,cemetery),f)).
believe_init(p,credible(at(r,family),f)).
locally_violated(s1,doNotLie).
totalUti(s1,env,0).
beliefbase(p).
totalUti(s1,p,0).
principle(doNotEnableMurder).
believe_init(p,murderer(f)).
believe_cons(p,wrongInfo(f,p)).
believe_cons(p,evade(p,f)).
believe_init(p,credible(at(r,madrid),f)).
principle(doNotLieExcept).
believe_cons(p,check(f,madrid,p)).
believe_init(p,at(r,family)).
impermissible(deontologism,s1).
act(s1,tell(p,at(r,madrid),f)).
beliefbase(env).
%AS2
permissible(principialism1,s2).
permissible(deontologism,s2).
totalUti(s2,env,0).
act(s2,silence(p,f)).
totalUti(s2,p,-2).
%AS3
act(s3,tell(p,at(r

In [58]:
# Run the combined program
answers_program2 = ASP(combined_program)
res = ""
# Output results for this specific answer set
for m, result in enumerate(answers_program2, start=1):
        res+= f"Answer Set {m} of program2 for Answer Set {n}:\n"
        for fact in result:
            res+=str(fact)+"\n"

print(res)

Answer Set 1 of program2 for Answer Set 4:
('permissible', ('consequentialism2', 's4'))
('totalUti', ('s3', 'p', -4))
('totalUti2', ('s3', 'env', 0))
('permissible', ('principialism1', 's2'))
('permissible', ('principialism1', 's1'))
('impermissible', ('deontologism', 's1'))
('impermissible', ('consequentialism2', 's1'))
('totalUti2', ('s3', 'p', -4))
('permissible', ('principialism2', 's4'))
('impermissible', ('deontologism', 's3'))
('impermissible', ('consequentialism1', 's2'))
('impermissible', ('consequentialism1', 's3'))
('totalUti', ('s2', 'env', 0))
('act', ('s3', 'tell(p,at(r,family),f)'))
('act', ('s2', 'silence(p,f)'))
('totalUti', ('s4', 'p', -4))
('permissible', ('deontologism', 's2'))
('act', ('s1', 'tell(p,at(r,madrid),f)'))
('permissible', ('consequentialism1', 's1'))
('totalUti2', ('s2', 'env', 0))
('impermissible', ('principialism2', 's3'))
('impermissible', ('consequentialism2', 's2'))
('totalUti', ('s1', 'p', 0))
('totalUti2', ('s4', 'p', -4))
('permissible', ('princ

In [59]:
def generate_markdown_table(answer_set_str):
    import re
    from collections import defaultdict
    acts = {
        'tell(p,at(r,madrid),f)': 'tell madrid',
        'tell(p,at(r,family),f)': 'tell family',
        'tell(p,at(r,cemetery),f)': 'tell cimetery',
        'silence(p,f)': 'silence'
    }
    theories = ['deontologism', 'principialism1', 'principialism2', 'consequentialism1', 'consequentialism2']
    dico_perm = dict()
    for e in theories:
        dico_perm[e] = []

    lines = answer_set_str.strip().split('\n')
    dico_act = dict()

    for line in lines:
      if line.startswith("('act'"):
        match = re.search(r"\('([^']+)', '([^']+)'\)", line)
        if match:
          scenario = match.group(1)
          action = match.group(2)
          dico_act[scenario] = acts.get(action, action)

      elif line.startswith("('permissible'"):
        match = re.search(r"\('([^']+)', '([^']+)'\)", line)
        if match:
          theory = match.group(1)
          scenario = match.group(2)
          dico_perm[theory].append((scenario,"perm"))
      elif line.startswith("('impermissible'"):
        match = re.search(r"\('([^']+)', '([^']+)'\)", line)
        if match:
          theory = match.group(1)
          scenario = match.group(2)
          dico_perm[theory].append((scenario,"imp"))

    print(dico_act)
    all_scenarios = set(dico_act.keys())
    mk = "| Act | " + " | ".join(theories) + " |\n"
    mk += "| :---------------: |" + " :-----: |" *len(theories) + "\n"

    for scenario in all_scenarios:
        row = f"| {dico_act[scenario]} "

        for theory in theories:
            for s, status in dico_perm[theory]:
                if s == scenario:
                    perm_status = status
                    break
            row += f"| {perm_status} "

        row += "|\n"
        mk += row

    return mk

markdown = generate_markdown_table(res)

{'s3': 'tell family', 's2': 'silence', 's1': 'tell madrid', 's4': 'tell cimetery'}


In [60]:
print(markdown)

| Act | deontologism | principialism1 | principialism2 | consequentialism1 | consequentialism2 |
| :---------------: | :-----: | :-----: | :-----: | :-----: | :-----: |
| tell family | imp | imp | imp | imp | imp |
| silence | perm | perm | perm | imp | imp |
| tell cimetery | imp | perm | perm | imp | perm |
| tell madrid | imp | perm | perm | perm | imp |



## Résultats

# situation 1
Falangiste meurtrier

| Act | deontologism | principialism1 | principialism2 | consequentialism1 | consequentialism2 |
| :---------------: | :-----: | :-----: | :-----: | :-----: | :-----: |
| tell madrid | imp | perm | perm | perm | perm |
| silence | perm | perm | perm | imp | imp |
| tell family | imp | imp | imp | imp | imp |
| tell cimetery | imp | perm | perm | imp | imp |


# situation 2
falangiste ni hostile ni meurtrier

| Act | deontologism | principialism1 | principialism2 | consequentialism1 | consequentialism2 |
| :---------------: | :-----: | :-----: | :-----: | :-----: | :-----: |
| tell cimetery | imp | imp | imp | imp | perm |
| silence | perm | perm | perm | imp | imp |
| tell madrid | imp | imp | imp | imp | imp |
| tell family | perm | perm | perm | perm | imp |


# situation 3
falangist ni meurtrier ni hostile mais pablo ne le sait pas

| Act | deontologism | principialism1 | principialism2 | consequentialism1 | consequentialism2 |
| :---------------: | :-----: | :-----: | :-----: | :-----: | :-----: |
| tell family | imp | imp | imp | imp | imp |
| silence | perm | perm | perm | imp | imp |
| tell cimetery | imp | perm | perm | imp | perm |
| tell madrid | imp | perm | perm | perm | imp |
